**worker**

## Live Gezichtsherkenning

In [ ]:
#RTSP_URL = "jouw_TRSP-datastream"

In [ ]:
import cv2
import os
from threading import Thread, Lock
from IPython.display import display
import ipywidgets as widgets
import time
import numpy as np
import matplotlib.pyplot as plt
from queue import Queue

In [ ]:
BASE_DIR = "../dataset" # of je eigen locatie
# open de camera
cap = cv2.VideoCapture(1)
lock = Lock()

## inladen LBPH-modellen

In [ ]:
# OpenCV haarcascade gezichtsdetectie
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# **Automatisch alle modellen laden**
models = {}
for person in os.listdir(BASE_DIR):
    model_path = os.path.join(BASE_DIR, person, f"face_model_{person}.yml")
    if os.path.exists(model_path):
        model = cv2.face.LBPHFaceRecognizer_create()
        model.read(model_path)
        models[person] = model
        print(f"Model geladen voor {person}")

if not models:
    print("Geen modellen correct geladen. Train eerst een model.")

# Widgets voor de live feed
image_widget = widgets.Image(format='jpeg')
stop_button = widgets.Button(description="Stop")
threshold_slider = widgets.IntSlider(value=100, min=50, max=150, step=1, description="Threshold")
output = widgets.Output()
display(threshold_slider, image_widget, stop_button, output)
running = True

# Houdt per persoon bij wanneer deze voor het laatst werd gegroet
last_greet = {}
GREETING_INTERVAL = 10  # seconden

# Creëer een queue voor spraakberichten
speech_queue = Queue()

# Deze worker verwerkt spraakberichten één voor één, zodat engine.runAndWait() niet overlapt
def speech_worker():
    while running:
        try:
            message = speech_queue.get(timeout=1)
        except Empty:
            continue
        engine.say(message)
        engine.runAndWait()
        speech_queue.task_done()

speech_thread = Thread(target=speech_worker, daemon=True)
speech_thread.start()

# **Live gezichtsdetectie en herkenning in een aparte thread**
def update_stream():
    global running
    while running:
        with lock:
            cap.grab()
            ret, frame = cap.read()
        
        if not ret:
            print("Geen frame ontvangen. Controleer de camera.")
            break

        # Converteer naar grijswaarden voor gezichtsdetectie
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
        
        # Verzamel in deze set alle unieke herkende personen in het huidige frame
        current_frame_names = set()

        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, (150, 150))  # Formaat voor herkenning

            best_name = "Unknown"
            best_confidence = float("inf")
            confidence_threshold = threshold_slider.value

            # Vergelijk gezicht met alle modellen
            for name, model in models.items():
                label, confidence = model.predict(face_resized)
                if confidence < best_confidence:
                    best_confidence = confidence
                    best_name = name
            
            # Teken groen kader en toon de naam als de confidence onder de threshold zit
            if best_confidence < confidence_threshold:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, best_name, (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                if best_name != "Unknown":
                    current_frame_names.add(best_name)
        
        # Voor iedere herkende persoon in dit frame, controleer de GREETING_INTERVAL
        current_time = time.time()
        for name in current_frame_names:
            if name not in last_greet or (current_time - last_greet[name]) > GREETING_INTERVAL:
                speech_queue.put(f"Hello, {name}, welcome home!")
                last_greet[name] = current_time

        # Toon live beeld in Jupyter
        _, buffer = cv2.imencode('.jpg', frame)
        image_widget.value = buffer.tobytes()
        time.sleep(0.03)

def stop_stream(_):
    global running
    running = False
    with lock:
        cap.release()

stop_button.on_click(stop_stream)

# Start de live stream in een aparte thread
thread = Thread(target=update_stream, daemon=True)
thread.start()
